- competition/dataset : [https://www.kaggle.com/c/porto-seguro-safe-driver-prediction](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction)
- date : 2021/01/27
- original : [https://www.kaggle.com/aharless/xgboost-cv-lb-284](https://www.kaggle.com/aharless/xgboost-cv-lb-284)

## XGBoost CV (LB .284)

**✏ 필사 1회** 

[oliver의 스크립트](https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283)를 따랐습니다.

In [1]:
MAX_ROUNDS = 400
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50
# 결정하는데 많은 정보를 사용하기 위해 EARLY_STOPPING_ROUNDS를 높게 설정
# early stopping을 원하면 EARLY_STOPPING_ROUNDS값을 줄여야 함

처음에는 ```MAX_ROUNDS```는 꽤 높게 설정하고 적절한 라운드의 수를 얻기 위해 ```OPTIMIZE_ROUNDS```를 사용하는 것을 추천합니다. (제 판단으로는 모든 fold 중 ```best_ntree_limit```의 최댓값에 근접해야 하며, 모델이 정규화된 경우 더 값이 클 것입니다. 혹은 ```verbose=True```를 설정하여 세부내용을 살펴보면서 모든 fold에 대해 잘 작동하는 라운드의 수를 찾을 수 있습니다.)  그런 다음, ```OPTIMIZE_ROUNDS```를 제거하고 ```MAX_ROUNDS```를 적절한 총 라운드 수로 설정합니다.  

각 fold에 가장 적합한 라운드를 선택하는 '조기 종료'는 검증 데이터에 과적합하는 문제가 있습니다. 따라서 테스트 데이터를 예측하기 위한 최적의 모델을 생성하지 못할 수 있고, 다른 모델들을 스태킹/앙상블하기 위해 검증 데이터를 생성할 경우 앙상블에 너무 많은 weight을 가지게 할 수 있습니다. 또 다른 가능성(XGBoost에서는 기본값)은 최적 라운드보다 조기 종료가 실제로 발생하는 라운드(개선 부족으로 인한 지연 포함)를 사용하는 것입니다. 이 방법은 과적합 문제(지연 시간이 너무 길어질 때의 문제점)를 해결해주지만, 현재까지 그다지 도움이 되지는 않았습니다. (모든 fold의 라운드 수를 사용했을 때보다 fold별로 20 라운드의 조기 종료를 사용했을 때 더 나쁜 검증 점수를 얻었습니다.)

In [2]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from numba import jit
import time
import gc
import warnings
warnings.filterwarnings('ignore')

gini 지수 계산 과정은 [CPMP의 커널 참고](https://www.kaggle.com/cpmpml/extremely-fast-gini-computation)  
$$gini = \sum_{i=1}^{d}{(R_{i}(1-\sum_{k=1}^{m}{p_{ik}^{2}}))}$$

In [3]:
# gini 계산
@jit
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2*gini / (ntrue*(n - ntrue))
    return gini

[oliver의 커널 참고](https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283)

In [4]:
def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = -evar_gini(labels, preds)
    return [('gini', gini_score)]

def add_noise(series, noise_level):
    return series * (1 + noise_level*np.random.randn(len(series)))

평활화는 [Daniele Micci-barreca](https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf)의 논문을 따랐습니다.  
* trn_series: Series 형태의 트레이닝 변수 (범주형)  
* tst_seires: Series 형태의 테스트 변수 (범주형)  
* target: Series 형태의 target 데이터  
* min_samples_leaf: 범주 평균을 고려한 최소 표본 수 (int)
* smoothing: 범주 평균과 이전 값의 균형을 위한 평활화 효과 (int)

In [5]:
def target_encode(trn_series=None,
                  val_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    
    # target의 평균값 계산
    averages = temp.groupby(trn_series.name)[target.name].agg(['mean', 'count'])
    
    # 평활화 계산
    smoothing = 1 / (1 + np.exp(-(averages['count']-min_samples_leaf)/smoothing))
    
    # 모든 target 데이터에 평균 함수 적용
    prior = target.mean()
    
    # count가 증가할수록 전체 평균 적게 고려
    averages[target.name] = prior*(1-smoothing) + averages['mean']*smoothing
    averages.drop(['mean', 'count'], axis=1, inplace=True)
    
    # trn, tst Series에 averages 적용
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index':target.name, target.name:'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)    
    # pd.merge는 인덱스를 유지하지 않으므로 목원
    ft_trn_series.index = trn_series.index
    
    ft_val_series = pd.merge(
        val_series.to_frame(val_series.name),
        averages.reset_index().rename(columns={'index':target.name, target.name:'average'}),
        on=val_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    ft_val_series.index = val_series.index
    
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index':target.name, target.name:'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_val_series, noise_level), add_noise(ft_tst_series, noise_level)

In [6]:
# 데이터 로딩
train_df = pd.read_csv('../data/porto_train.csv', na_values='-1')
test_df = pd.read_csv('../data/porto_test.csv', na_values='-1')

In [7]:
# from olivier
train_features = [
    "ps_car_13",      # : 1571.65 / shadow  609.23
    "ps_reg_03",      # : 1408.42 / shadow  511.15
    "ps_ind_05_cat",  # : 1387.87 / shadow   84.72
    "ps_ind_03",      # : 1219.47 / shadow  230.55
    "ps_ind_15",      # :  922.18 / shadow  242.00
    "ps_reg_02",      # :  920.65 / shadow  267.50
    "ps_car_14",      # :  798.48 / shadow  549.58
    "ps_car_12",      # :  731.93 / shadow  293.62
    "ps_car_01_cat",  # :  698.07 / shadow  178.72
    "ps_car_07_cat",  # :  694.53 / shadow   36.35
    "ps_ind_17_bin",  # :  620.77 / shadow   23.15
    "ps_car_03_cat",  # :  611.73 / shadow   50.67
    "ps_reg_01",      # :  598.60 / shadow  178.57
    "ps_car_15",      # :  593.35 / shadow  226.43
    "ps_ind_01",      # :  547.32 / shadow  154.58
    "ps_ind_16_bin",  # :  475.37 / shadow   34.17
    "ps_ind_07_bin",  # :  435.28 / shadow   28.92
    "ps_car_06_cat",  # :  398.02 / shadow  212.43
    "ps_car_04_cat",  # :  376.87 / shadow   76.98
    "ps_ind_06_bin",  # :  370.97 / shadow   36.13
    "ps_car_09_cat",  # :  214.12 / shadow   81.38
    "ps_car_02_cat",  # :  203.03 / shadow   26.67
    "ps_ind_02_cat",  # :  189.47 / shadow   65.68
    "ps_car_11",      # :  173.28 / shadow   76.45
    "ps_car_05_cat",  # :  172.75 / shadow   62.92
    "ps_calc_09",     # :  169.13 / shadow  129.72
    "ps_calc_05",     # :  148.83 / shadow  120.68
    "ps_ind_08_bin",  # :  140.73 / shadow   27.63
    "ps_car_08_cat",  # :  120.87 / shadow   28.82
    "ps_ind_09_bin",  # :  113.92 / shadow   27.05
    "ps_ind_04_cat",  # :  107.27 / shadow   37.43
    "ps_ind_18_bin",  # :   77.42 / shadow   25.97
    "ps_ind_12_bin",  # :   39.67 / shadow   15.52
    "ps_ind_14",      # :   37.37 / shadow   16.65
]
# 조합 추가
combs = [
    ('ps_reg_01', 'ps_car_02_cat'),  
    ('ps_reg_01', 'ps_car_04_cat'),
]

In [8]:
# 데이터 처리
id_test = test_df['id'].values
id_train = train_df['id'].values
y = train_df['target']

start = time.time()
for n_c, (f1, f2) in enumerate(combs):
    name1 = f1 + '_plus_' + f2
    print('current feature %60s %4d in %5.1f'%(name1, n_c+1, (time.time()-start)/60), end='')
    print('\r'*75, end='')
    
    train_df[name1] = train_df[f1].apply(lambda x: str(x)) + '_' + train_df[f2].apply(lambda x:str(x))
    test_df[name1] = test_df[f1].apply(lambda x: str(x)) + '_' + test_df[f2].apply(lambda x: str(x))
    
    # Label Encode
    lbl = LabelEncoder()
    lbl.fit(list(train_df[name1].values) + list(test_df[name1].values))
    train_df[name1] = lbl.transform(list(train_df[name1].values))
    test_df[name1] = lbl.transform(list(test_df[name1].values))
    
    train_features.append(name1)

x = train_df[train_features]
test_df = test_df[train_features]

f_cats = [f for f in x.columns if '_cat' in f]

current feature                                 ps_reg_01_plus_ps_car_02_cat    1 in   0.0current feature                                 ps_reg_01_plus_ps_car_04_cat    2 in   0.0

In [9]:
y_valid_pred = 0 * y
y_test_pred = 0

In [10]:
# folds 설정
K = 5
kf = KFold(n_splits=K, random_state=1, shuffle=True)
np.random.seed(0)

In [11]:
# 분류기 설정
model = XGBClassifier(
    n_estimators=MAX_ROUNDS,
    max_depth=4,
    objective='binary:logistic',
    learning_rate=LEARNING_RATE,
    subsample=.8,
    min_child_weight=6,
    colsample_bytree=.8,
    scale_pos_weight=1.6,
    gamma=10,
    reg_alpha=8,
    reg_lambda=1.3
)

In [28]:
for i, (train_index, test_index) in enumerate(kf.split(train_df)):
    
    # fold에 데이터 생성
    y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
    x_train, x_valid = x.iloc[train_index,:].copy(), x.iloc[test_index,:].copy()
    x_test = test_df.copy()
    print( "\nFold ", i)
    
    # 데이터 encoding
    for f in f_cats:
        x_train[f + "_avg"], x_valid[f + "_avg"], x_test[f + "_avg"] = target_encode(
            
            trn_series=x_train[f],
            val_series=x_valid[f],
            tst_series=x_test[f],
            target=y_train,
            min_samples_leaf=200,
            smoothing=10,
            noise_level=0
    )
    # Run model for this fold
    if OPTIMIZE_ROUNDS:
        eval_set=[(x_valid,y_valid)]
        fit_model = model.fit(
            x_train, y_train,
            eval_set=eval_set,
            eval_metric=gini_xgb,
            early_stopping_rounds=EARLY_STOPPING_ROUNDS,
            verbose=False
    )
        print("  Best N trees = ", model.best_ntree_limit)
        print("  Best gini = ", model.best_score)
    else:
        fit_model = model.fit(x_train, y_train)
        
    # 검증 예측 생성
    pred = fit_model.predict_proba(x_valid)[:, 1]
    print("  Gini = ", eval_gini(y_valid, pred))
    y_valid_pred.iloc[test_index] = pred
    
    # 테스트 셋 예측 축적
    y_test_pred += fit_model.predict_proba(x_test)[:, 1]
    
    del x_test, x_train, x_valid, y_train
    
y_test_pred /= K  # Average test set predictions

# 테스트 셋 예측값의 평균
print( "\nGini for full training set:" )
eval_gini(y, y_valid_pred)


Fold  0
[14:42:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
  Gini =  0.2851059728784705

Fold  1
[14:44:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
  Gini =  0.28185495483845957

Fold  2
[14:47:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore th

0.28501477642381845

In [29]:
# 스태킹/앙상블을 검증 예측 결과 저장
val = pd.DataFrame()
val['id'] = id_train
val['target'] = y_valid_pred.values
val.to_csv('../data/porto_3_xgb_valid.csv', float_format='%.6f', index=False)

In [30]:
# 제출 파일 생성
sub = pd.DataFrame()
sub['id'] = id_test
sub['target'] = y_test_pred
sub.to_csv('../data/porto_3_xgb_submit.csv', float_format='%.6f', index=False)

## XGBoost CV (LB .284)

**✏ 필사 2회** 

[oliver의 스크립트](https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283)를 따랐습니다.

In [1]:
MAX_ROUNDS = 400
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50
# 결정하는데 많은 정보를 사용하기 위해 EARLY_STOPPING_ROUNDS를 높게 설정
# early stopping을 원하면 EARLY_STOPPING_ROUNDS값을 줄여야 함

처음에는 ```MAX_ROUNDS```는 꽤 높게 설정하고 적절한 라운드의 수를 얻기 위해 ```OPTIMIZE_ROUNDS```를 사용하는 것을 추천합니다. (제 판단으로는 모든 fold 중 ```best_ntree_limit```의 최댓값에 근접해야 하며, 모델이 정규화된 경우 더 값이 클 것입니다. 혹은 ```verbose=True```를 설정하여 세부내용을 살펴보면서 모든 fold에 대해 잘 작동하는 라운드의 수를 찾을 수 있습니다.)  그런 다음, ```OPTIMIZE_ROUNDS```를 제거하고 ```MAX_ROUNDS```를 적절한 총 라운드 수로 설정합니다.  

각 fold에 가장 적합한 라운드를 선택하는 '조기 종료'는 검증 데이터에 과적합하는 문제가 있습니다. 따라서 테스트 데이터를 예측하기 위한 최적의 모델을 생성하지 못할 수 있고, 다른 모델들을 스태킹/앙상블하기 위해 검증 데이터를 생성할 경우 앙상블에 너무 많은 weight을 가지게 할 수 있습니다. 또 다른 가능성(XGBoost에서는 기본값)은 최적 라운드보다 조기 종료가 실제로 발생하는 라운드(개선 부족으로 인한 지연 포함)를 사용하는 것입니다. 이 방법은 과적합 문제(지연 시간이 너무 길어질 때의 문제점)를 해결해주지만, 현재까지 그다지 도움이 되지는 않았습니다. (모든 fold의 라운드 수를 사용했을 때보다 fold별로 20 라운드의 조기 종료를 사용했을 때 더 나쁜 검증 점수를 얻었습니다.)

In [2]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from numba import jit
import time
import gc
import warnings
warnings.filterwarnings('ignore')

gini 지수 계산 과정은 [CPMP의 커널 참고](https://www.kaggle.com/cpmpml/extremely-fast-gini-computation)  
$$gini = \sum_{i=1}^{d}{(R_{i}(1-\sum_{k=1}^{m}{p_{ik}^{2}}))}$$

In [3]:
# gini 계산
@jit
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2*gini / (ntrue*(n - ntrue))
    return gini

[oliver의 커널 참고](https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283)

In [4]:
def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = -evar_gini(labels, preds)
    return [('gini', gini_score)]

def add_noise(series, noise_level):
    return series * (1 + noise_level*np.random.randn(len(series)))

평활화는 [Daniele Micci-barreca](https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf)의 논문을 따랐습니다.  
* trn_series: Series 형태의 트레이닝 변수 (범주형)  
* tst_seires: Series 형태의 테스트 변수 (범주형)  
* target: Series 형태의 target 데이터  
* min_samples_leaf: 범주 평균을 고려한 최소 표본 수 (int)
* smoothing: 범주 평균과 이전 값의 균형을 위한 평활화 효과 (int)

In [5]:
def target_encode(trn_series=None,
                  val_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    
    # target의 평균값 계산
    averages = temp.groupby(trn_series.name)[target.name].agg(['mean', 'count'])
    
    # 평활화 계산
    smoothing = 1 / (1 + np.exp(-(averages['count']-min_samples_leaf)/smoothing))
    
    # 모든 target 데이터에 평균 함수 적용
    prior = target.mean()
    
    # count가 증가할수록 전체 평균 적게 고려
    averages[target.name] = prior*(1-smoothing) + averages['mean']*smoothing
    averages.drop(['mean', 'count'], axis=1, inplace=True)
    
    # trn, tst Series에 averages 적용
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index':target.name, target.name:'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)    
    # pd.merge는 인덱스를 유지하지 않으므로 목원
    ft_trn_series.index = trn_series.index
    
    ft_val_series = pd.merge(
        val_series.to_frame(val_series.name),
        averages.reset_index().rename(columns={'index':target.name, target.name:'average'}),
        on=val_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    ft_val_series.index = val_series.index
    
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index':target.name, target.name:'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_val_series, noise_level), add_noise(ft_tst_series, noise_level)

In [6]:
# 데이터 로딩
train_df = pd.read_csv('../data/porto_train.csv', na_values='-1')
test_df = pd.read_csv('../data/porto_test.csv', na_values='-1')

In [7]:
# from olivier
train_features = [
    "ps_car_13",      # : 1571.65 / shadow  609.23
    "ps_reg_03",      # : 1408.42 / shadow  511.15
    "ps_ind_05_cat",  # : 1387.87 / shadow   84.72
    "ps_ind_03",      # : 1219.47 / shadow  230.55
    "ps_ind_15",      # :  922.18 / shadow  242.00
    "ps_reg_02",      # :  920.65 / shadow  267.50
    "ps_car_14",      # :  798.48 / shadow  549.58
    "ps_car_12",      # :  731.93 / shadow  293.62
    "ps_car_01_cat",  # :  698.07 / shadow  178.72
    "ps_car_07_cat",  # :  694.53 / shadow   36.35
    "ps_ind_17_bin",  # :  620.77 / shadow   23.15
    "ps_car_03_cat",  # :  611.73 / shadow   50.67
    "ps_reg_01",      # :  598.60 / shadow  178.57
    "ps_car_15",      # :  593.35 / shadow  226.43
    "ps_ind_01",      # :  547.32 / shadow  154.58
    "ps_ind_16_bin",  # :  475.37 / shadow   34.17
    "ps_ind_07_bin",  # :  435.28 / shadow   28.92
    "ps_car_06_cat",  # :  398.02 / shadow  212.43
    "ps_car_04_cat",  # :  376.87 / shadow   76.98
    "ps_ind_06_bin",  # :  370.97 / shadow   36.13
    "ps_car_09_cat",  # :  214.12 / shadow   81.38
    "ps_car_02_cat",  # :  203.03 / shadow   26.67
    "ps_ind_02_cat",  # :  189.47 / shadow   65.68
    "ps_car_11",      # :  173.28 / shadow   76.45
    "ps_car_05_cat",  # :  172.75 / shadow   62.92
    "ps_calc_09",     # :  169.13 / shadow  129.72
    "ps_calc_05",     # :  148.83 / shadow  120.68
    "ps_ind_08_bin",  # :  140.73 / shadow   27.63
    "ps_car_08_cat",  # :  120.87 / shadow   28.82
    "ps_ind_09_bin",  # :  113.92 / shadow   27.05
    "ps_ind_04_cat",  # :  107.27 / shadow   37.43
    "ps_ind_18_bin",  # :   77.42 / shadow   25.97
    "ps_ind_12_bin",  # :   39.67 / shadow   15.52
    "ps_ind_14",      # :   37.37 / shadow   16.65
]
# 조합 추가
combs = [
    ('ps_reg_01', 'ps_car_02_cat'),  
    ('ps_reg_01', 'ps_car_04_cat'),
]

In [8]:
# 데이터 처리
id_test = test_df['id'].values
id_train = train_df['id'].values
y = train_df['target']

start = time.time()
for n_c, (f1, f2) in enumerate(combs):
    name1 = f1 + '_plus_' + f2
    print('current feature %60s %4d in %5.1f'%(name1, n_c+1, (time.time()-start)/60), end='')
    print('\r'*75, end='')
    
    train_df[name1] = train_df[f1].apply(lambda x: str(x)) + '_' + train_df[f2].apply(lambda x:str(x))
    test_df[name1] = test_df[f1].apply(lambda x: str(x)) + '_' + test_df[f2].apply(lambda x: str(x))
    
    # Label Encode
    lbl = LabelEncoder()
    lbl.fit(list(train_df[name1].values) + list(test_df[name1].values))
    train_df[name1] = lbl.transform(list(train_df[name1].values))
    test_df[name1] = lbl.transform(list(test_df[name1].values))
    
    train_features.append(name1)

x = train_df[train_features]
test_df = test_df[train_features]

f_cats = [f for f in x.columns if '_cat' in f]

current feature                                 ps_reg_01_plus_ps_car_02_cat    1 in   0.0current feature                                 ps_reg_01_plus_ps_car_04_cat    2 in   0.0

In [9]:
y_valid_pred = 0 * y
y_test_pred = 0

In [10]:
# folds 설정
K = 5
kf = KFold(n_splits=K, random_state=1, shuffle=True)
np.random.seed(0)

In [11]:
# 분류기 설정
model = XGBClassifier(
    n_estimators=MAX_ROUNDS,
    max_depth=4,
    objective='binary:logistic',
    learning_rate=LEARNING_RATE,
    subsample=.8,
    min_child_weight=6,
    colsample_bytree=.8,
    scale_pos_weight=1.6,
    gamma=10,
    reg_alpha=8,
    reg_lambda=1.3
)

In [12]:
for i, (train_index, test_index) in enumerate(kf.split(train_df)):
    
    # fold에 데이터 생성
    y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
    x_train, x_valid = x.iloc[train_index,:].copy(), x.iloc[test_index,:].copy()
    x_test = test_df.copy()
    print( "\nFold ", i)
    
    # 데이터 encoding
    for f in f_cats:
        x_train[f + "_avg"], x_valid[f + "_avg"], x_test[f + "_avg"] = target_encode(
            
            trn_series=x_train[f],
            val_series=x_valid[f],
            tst_series=x_test[f],
            target=y_train,
            min_samples_leaf=200,
            smoothing=10,
            noise_level=0
    )
    # Run model for this fold
    if OPTIMIZE_ROUNDS:
        eval_set=[(x_valid,y_valid)]
        fit_model = model.fit(
            x_train, y_train,
            eval_set=eval_set,
            eval_metric=gini_xgb,
            early_stopping_rounds=EARLY_STOPPING_ROUNDS,
            verbose=False
    )
        print("  Best N trees = ", model.best_ntree_limit)
        print("  Best gini = ", model.best_score)
    else:
        fit_model = model.fit(x_train, y_train)
        
    # 검증 예측 생성
    pred = fit_model.predict_proba(x_valid)[:, 1]
    print("  Gini = ", eval_gini(y_valid, pred))
    y_valid_pred.iloc[test_index] = pred
    
    # 테스트 셋 예측 축적
    y_test_pred += fit_model.predict_proba(x_test)[:, 1]
    
    del x_test, x_train, x_valid, y_train
    
y_test_pred /= K  # Average test set predictions

# 테스트 셋 예측값의 평균
print( "\nGini for full training set:" )
eval_gini(y, y_valid_pred)


Fold  0
[18:16:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
  Gini =  0.2851059728784705

Fold  1
[18:18:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
  Gini =  0.28185495483845957

Fold  2
[18:20:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore th

0.28501477642381845

In [13]:
# 스태킹/앙상블을 검증 예측 결과 저장
val = pd.DataFrame()
val['id'] = id_train
val['target'] = y_valid_pred.values
val.to_csv('../data/porto_3_xgb_valid.csv', float_format='%.6f', index=False)

In [14]:
# 제출 파일 생성
sub = pd.DataFrame()
sub['id'] = id_test
sub['target'] = y_test_pred
sub.to_csv('../data/porto_3_xgb_submit.csv', float_format='%.6f', index=False)